# 线性回归的简洁实现
如何(**通过使用深度学习框架来简洁地实现**)
 :numref:`sec_linear_scratch`中的(**线性回归模型**)。

## 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 读取数据集

[**调用框架中现有的API来读取数据**]。
将`features`和`labels`作为API的参数传递，并通过数据迭代器指定`batch_size`。
此外，布尔值`is_train`表示是否希望数据迭代器对象在每个迭代周期内打乱数据。


In [3]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [4]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

为了验证是否正常工作，读取并打印第一个小批量样本：使用iter构造Python迭代器，并使用next从迭代器中获取第一项。

In [5]:
next(iter(data_iter))

[tensor([[ 0.2852,  1.1958],
         [ 0.2385, -1.7974],
         [-1.7904, -1.4159],
         [ 0.0225, -0.7400],
         [ 2.1171, -0.4587],
         [-0.8494, -1.5717],
         [ 0.3191, -0.1975],
         [ 0.4305, -0.5682],
         [-0.0098, -0.0663],
         [-0.8820,  1.7442]]),
 tensor([[ 0.7039],
         [10.7877],
         [ 5.4310],
         [ 6.7736],
         [10.0091],
         [ 7.8271],
         [ 5.5065],
         [ 6.9909],
         [ 4.3963],
         [-3.5062]])]

## 定义模型
*全连接层*（fully-connected layer）：每一个输入都通过矩阵-向量乘法得到它的每个输出。  
在PyTorch中，全连接层在`Linear`类中定义，我们将两个参数传递到`nn.Linear`中。
第一个指定输入特征形状，第二个指定输出特征形状。

In [6]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

## 初始化模型参数
同构造`nn.Linear`时指定输入和输出尺寸一样，能直接访问参数以设定它们的初始值。  
通过`net[0]`选择网络中的第一个图层，然后使用`weight.data`和`bias.data`方法访问参数。  
并且还可以使用替换方法`normal_`和`fill_`来重写参数值。

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 定义损失函数
[**计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数**]。
默认情况下，它返回所有样本损失的平均值。

In [8]:
loss = nn.MSELoss()

## 定义优化算法
小批量随机梯度下降算法是一种优化神经网络的标准工具，PyTorch在`optim`模块中实现了该算法的许多变种。  
当(**实例化一个`SGD`实例**)时，要指定优化的参数：`net.parameters()`以及优化算法所需的超参数字典如小批量随机梯度下降需要设置`lr`值。

In [10]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 训练
在每个迭代周期里，将完整遍历一次数据集（`train_data`），
不停地从中获取一个小批量的输入和相应的标签。
对于每一个小批量，会进行以下步骤:

* 通过调用`net(X)`生成预测并计算损失`l`（前向传播）。
* 通过进行反向传播来计算梯度。
* 通过调用优化器来更新模型参数。

为了更好的衡量训练效果，需要计算每个迭代周期后的损失，并打印它来监控训练过程。


In [11]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000241
epoch 2, loss 0.000097
epoch 3, loss 0.000097


In [12]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-2.6608e-04,  4.0531e-05])
b的估计误差： tensor([-0.0002])
